In [119]:
import pandas as pd
import numpy as np
from WrongData import introduce_filthy_data
from sklearn.impute import KNNImputer

## Adding filthy data

In [120]:
df = pd.read_csv("./dataset/spotify_songs_restructured.csv", sep=",")
new_data = introduce_filthy_data(df, 1500)
new_data.to_csv("./dataset/spotify_songs_restructured_filthy.csv", index=False)
new_data.describe()

,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
count,31151.000000,31194.000000,31120.000000,31178.000000,31206.000000,31195.000000,31210.000000,31152.000000,31260.000000,31232.000000,31211.000000,31173.000000,31169.000000
mean,42.508651,0.654900,0.698783,5.368305,-6.719323,0.565475,0.106960,0.175698,0.083990,0.190316,0.510438,120.869491,225662.600789
std,24.992782,0.144964,0.180787,3.614688,2.978744,0.495702,0.101047,0.220059,0.223059,0.154512,0.233377,26.900344,59712.202009
min,0.000000,0.000000,0.000175,0.000000,-46.448000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4000.000000
25%,24.000000,0.563000,0.582000,2.000000,-8.179000,0.000000,0.041000,0.015100,0.000000,0.092700,0.330000,99.957000,187800.000000
50%,45.000000,0.672000,0.722000,6.000000,-6.168000,1.000000,0.062600,0.080500,0.000016,0.127000,0.512000,121.982000,216000.000000
75%,62.000000,0.760000,0.840000,9.000000,-4.648250,1.000000,0.131000,0.255250,0.004752,0.248000,0.693000,133.879000,253333.000000
max,100.000000,0.983000,1.000000,11.000000,1.275000,1.000000,0.918000,0.994000,0.994000,0.996000,0.991000,239.440000,517810.000000


In [121]:
new_data

,track_name,track_artist,track_popularity,track_album_name,track_album_release_date,playlist_name,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66.0,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,pop,dance pop,0.748,0.916,6.0,-2.634,1.0,0.0583,0.102000,0.000000,0.0653,NaN,122.036,194754.0
1,Memories - Dillon Francis Remix,Maroon 5,67.0,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,pop,dance pop,0.726,0.815,11.0,-4.969,1.0,0.0373,0.072400,0.004210,0.3570,NaN,99.972,162600.0
2,All the Time - Don Diablo Remix,Zara Larsson,NaN,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,pop,dance pop,0.675,0.931,1.0,-3.432,0.0,0.0742,0.079400,0.000023,0.1100,0.613,124.008,176616.0
3,NaN,The Chainsmokers,60.0,Call You Mine - The Remixes,2019-07-19,Pop Remix,pop,dance pop,0.718,0.930,7.0,-3.778,1.0,0.1020,0.028700,0.000009,0.2040,0.277,121.956,169093.0
4,Someone You Loved - Future Humans Remix,Lewis Capaldi,69.0,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,pop,dance pop,0.650,0.833,1.0,-4.672,1.0,0.0359,0.080300,0.000000,0.0833,NaN,123.976,189052.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32828,City Of Lights - Official Radio Edit,Lush & Simon,42.0,City Of Lights (Vocal Mix),2014-04-28,♥ EDM LOVE 2020,edm,progressive electro house,0.428,0.922,NaN,-1.814,1.0,0.0936,0.076600,0.000000,0.0668,0.210,128.170,204375.0
32829,Closer - Sultan & Ned Shepard Remix,Tegan and Sara,20.0,Closer Remixed,NaN,♥ EDM LOVE 2020,edm,progressive electro house,0.522,0.786,0.0,-4.462,1.0,0.0420,0.001710,0.004270,0.3750,0.400,128.041,353120.0
32830,Sweet Surrender - Radio Edit,Starkillers,14.0,Sweet Surrender (Radio Edit),2014-04-21,♥ EDM LOVE 2020,edm,progressive electro house,0.529,0.821,6.0,NaN,0.0,0.0481,0.108000,0.000001,0.1500,0.436,127.989,210112.0
32831,Only For You - Maor Levi Remix,NaN,15.0,Only For You (Remixes),2014-01-01,♥ EDM LOVE 2020,edm,progressive electro house,0.626,0.888,2.0,-3.361,1.0,0.1090,0.007920,0.127000,0.3430,0.308,128.008,367432.0


## Transforming the data

In [122]:
# nan van naam weghalen
df_clean = new_data.dropna(subset=['track_name', 'track_artist', 'track_album_name', 'track_album_release_date', 'playlist_name',
                                   'playlist_genre', 'playlist_subgenre'])
# df_clean = new_data.dropna(subset=['track_album_release_date'])
df_clean.head(10)

,track_name,track_artist,track_popularity,track_album_name,track_album_release_date,playlist_name,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66.0,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,pop,dance pop,0.748,0.916,6.0,-2.634,1.0,0.0583,0.1020,0.000000,0.0653,NaN,122.036,194754.0
1,Memories - Dillon Francis Remix,Maroon 5,67.0,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,pop,dance pop,0.726,0.815,11.0,-4.969,1.0,0.0373,0.0724,0.004210,0.3570,NaN,99.972,162600.0
2,All the Time - Don Diablo Remix,Zara Larsson,NaN,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,pop,dance pop,0.675,0.931,1.0,-3.432,0.0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616.0
4,Someone You Loved - Future Humans Remix,Lewis Capaldi,69.0,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,pop,dance pop,0.650,0.833,1.0,-4.672,1.0,0.0359,0.0803,0.000000,0.0833,NaN,123.976,189052.0
5,Beautiful People (feat. Khalid) - Jack Wins Remix,Ed Sheeran,67.0,Beautiful People (feat. Khalid) [Jack Wins Remix],2019-07-11,Pop Remix,pop,dance pop,0.675,NaN,8.0,-5.385,1.0,0.1270,0.0799,0.000000,0.1430,0.585,124.982,163049.0
6,Never Really Over - R3HAB Remix,Katy Perry,NaN,Never Really Over (R3HAB Remix),2019-07-26,Pop Remix,pop,dance pop,0.449,0.856,5.0,-4.788,0.0,0.0623,0.1870,0.000000,0.1760,0.152,112.648,187675.0
7,Post Malone (feat. RANI) - GATTÜSO Remix,Sam Feldt,69.0,Post Malone (feat. RANI) [GATTÜSO Remix],2019-08-29,Pop Remix,pop,dance pop,0.542,0.903,NaN,-2.419,0.0,0.0434,0.0335,0.000005,0.1110,0.367,127.936,207619.0
8,Tough Love - Tiësto Remix / Radio Edit,Avicii,68.0,Tough Love (Tiësto Remix),2019-06-14,Pop Remix,pop,dance pop,0.594,0.935,8.0,-3.562,1.0,0.0565,0.0249,0.000004,0.6370,NaN,127.015,193187.0
9,If I Can't Have You - Gryffin Remix,Shawn Mendes,67.0,If I Can't Have You (Gryffin Remix),2019-06-20,Pop Remix,pop,dance pop,0.642,0.818,NaN,-4.552,1.0,0.0320,0.0567,0.000000,0.0919,0.590,124.957,253040.0
10,Cross Me (feat. Chance the Rapper & PnB Rock) ...,Ed Sheeran,NaN,Cross Me (feat. Chance the Rapper & PnB Rock) ...,2019-06-21,Pop Remix,pop,dance pop,0.679,0.923,6.0,-6.500,1.0,0.1810,0.1460,0.000005,0.1240,0.752,121.984,207894.0


### Imputing missing values

In [123]:
df_clean = df_clean.fillna({
    "danceability": df_clean["danceability"].mean(), 
    "energy": df_clean.energy.mean(), 
    "key": df_clean.key.mean(), 
    "loudness": df_clean.loudness.mean(), 
    "mode": df_clean['mode'].mean(), 
    "speechiness": df_clean.speechiness.mean(), 
    "acousticness": df_clean.acousticness.mean(), 
    "instrumentalness": df_clean.instrumentalness.mean(), 
    "liveness": df_clean.liveness.mean(), 
    "valence": df_clean.valence.mean(), 
    "tempo": df_clean.tempo.mean(), 
    "duration_ms": df_clean.duration_ms.mean(),
    "track_popularity": df_clean.track_popularity.mean()
})

df_clean.describe()

,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
count,21376.000000,21376.000000,21376.000000,21376.000000,21376.000000,21376.000000,21376.000000,21376.000000,21376.000000,21376.000000,21376.000000,21376.000000,21376.000000
mean,42.456890,0.654565,0.698571,5.386847,-6.722152,0.561100,0.106711,0.175390,0.083127,0.190546,0.509883,120.838811,225705.772282
std,23.986481,0.138479,0.173557,3.466827,2.877055,0.476872,0.096502,0.210934,0.213119,0.149864,0.224284,25.918583,56994.287122
min,0.000000,0.098500,0.008140,0.000000,-46.448000,0.000000,0.022400,0.000001,0.000000,0.009360,0.000010,37.114000,31893.000000
25%,27.000000,0.573000,0.596000,2.000000,-7.958000,0.000000,0.042200,0.018000,0.000000,0.095200,0.345000,100.086750,190412.750000
50%,42.456890,0.655000,0.700000,5.386847,-6.440000,1.000000,0.067900,0.099650,0.000035,0.137000,0.509883,120.838811,221010.500000
75%,61.000000,0.752000,0.830000,8.000000,-4.780000,1.000000,0.121000,0.231000,0.023225,0.232000,0.678000,130.159000,249188.000000
max,100.000000,0.983000,1.000000,11.000000,1.135000,1.000000,0.869000,0.992000,0.994000,0.996000,0.990000,239.440000,517125.000000


In [124]:
df_clean

,track_name,track_artist,track_popularity,track_album_name,track_album_release_date,playlist_name,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66.00000,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,pop,dance pop,0.748,0.916000,6.000000,-2.634000,1.0,0.058300,0.102000,0.000000,0.0653,0.509883,122.036,194754.0
1,Memories - Dillon Francis Remix,Maroon 5,67.00000,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,pop,dance pop,0.726,0.815000,11.000000,-4.969000,1.0,0.037300,0.072400,0.004210,0.3570,0.509883,99.972,162600.0
2,All the Time - Don Diablo Remix,Zara Larsson,42.45689,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,pop,dance pop,0.675,0.931000,1.000000,-3.432000,0.0,0.074200,0.079400,0.000023,0.1100,0.613000,124.008,176616.0
4,Someone You Loved - Future Humans Remix,Lewis Capaldi,69.00000,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,pop,dance pop,0.650,0.833000,1.000000,-4.672000,1.0,0.035900,0.080300,0.000000,0.0833,0.509883,123.976,189052.0
5,Beautiful People (feat. Khalid) - Jack Wins Remix,Ed Sheeran,67.00000,Beautiful People (feat. Khalid) [Jack Wins Remix],2019-07-11,Pop Remix,pop,dance pop,0.675,0.698571,8.000000,-5.385000,1.0,0.127000,0.079900,0.000000,0.1430,0.585000,124.982,163049.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32826,Wasted,Tiësto,42.45689,Wasted,2014-04-22,♥ EDM LOVE 2020,edm,progressive electro house,0.645,0.832000,2.000000,-5.595000,1.0,0.029400,0.001060,0.002640,0.1990,0.375000,112.028,188371.0
32827,Many Ways - Radio Edit,Ferry Corsten feat. Jenny Wahlstrom,27.00000,Many Ways,2013,♥ EDM LOVE 2020,edm,progressive electro house,0.581,0.640000,5.000000,-8.367000,1.0,0.106711,0.026600,0.000000,0.5720,0.288000,128.001,196993.0
32828,City Of Lights - Official Radio Edit,Lush & Simon,42.00000,City Of Lights (Vocal Mix),2014-04-28,♥ EDM LOVE 2020,edm,progressive electro house,0.428,0.922000,5.386847,-1.814000,1.0,0.093600,0.076600,0.000000,0.0668,0.210000,128.170,204375.0
32830,Sweet Surrender - Radio Edit,Starkillers,14.00000,Sweet Surrender (Radio Edit),2014-04-21,♥ EDM LOVE 2020,edm,progressive electro house,0.529,0.821000,6.000000,-6.722152,0.0,0.048100,0.108000,0.000001,0.1500,0.436000,127.989,210112.0


### outliers removal

In [125]:
# Define the columns where you want to remove outliers
columns_to_clean = ["danceability", "energy", "key", "loudness", "mode", 
                    "speechiness", "acousticness", "instrumentalness", 
                    "liveness", "valence", "tempo", "duration_ms"]

# Loop through each column and apply the custom quantile-based outlier removal
for col in columns_to_clean:
    Q01 = df_clean[col].quantile(0.001)  # 0.1st percentile
    Q99 = df_clean[col].quantile(0.999)  # 99.9th percentile
    
    # Remove rows where the values in the column are outside the 1st and 99th percentile bounds
    df_clean = df_clean[(df_clean[col] >= Q01) & (df_clean[col] <= Q99)]

# Display the cleaned DataFrame
df_clean.describe()

,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
count,20970.000000,20970.000000,20970.000000,20970.000000,20970.000000,20970.000000,20970.000000,20970.000000,20970.000000,20970.000000,20970.000000,20970.000000,20970.000000
mean,42.544820,0.655754,0.700143,5.385418,-6.675162,0.560697,0.106344,0.173166,0.079154,0.189863,0.510701,120.824719,225684.075018
std,24.014587,0.136866,0.170145,3.469171,2.750361,0.476897,0.094576,0.207031,0.206579,0.147298,0.222935,25.635962,55668.825529
min,0.000000,0.168000,0.075000,0.000000,-20.376000,0.000000,0.023900,0.000008,0.000000,0.021900,0.035000,62.520000,73813.000000
25%,27.000000,0.574000,0.597000,2.000000,-7.924000,0.000000,0.042200,0.018100,0.000000,0.095200,0.346000,100.116500,190694.250000
50%,42.456890,0.656000,0.701000,5.386847,-6.425000,1.000000,0.068000,0.099000,0.000031,0.137000,0.509883,120.838811,220940.500000
75%,61.000000,0.752000,0.830000,8.000000,-4.773000,1.000000,0.121000,0.229000,0.019700,0.232000,0.677000,130.104500,248916.750000
max,100.000000,0.964000,0.995000,11.000000,-0.955000,1.000000,0.609000,0.968000,0.955000,0.973000,0.973000,205.958000,504200.000000


### Categorical Encoding

In [126]:
# Corrected bins for track_popularity (7 edges, 6 labels)
popularity_bins = [0, 10, 30, 50, 70, 90, 100]
popularity_labels = ['Very Low Popularity', 'Low Popularity', 'Medium-Low Popularity', 'Medium Popularity', 'High Popularity', 'Very High Popularity']
df_clean['popularity_category'] = pd.Categorical(pd.cut(df_clean['track_popularity'], bins=popularity_bins, labels=popularity_labels))

# Corrected bins for tempo (8 edges, 7 labels)
tempo_bins = [60, 80, 100, 120, 140, 160, 180, 205]
tempo_labels = ['Very Slow', 'Slow', 'Moderate', 'Fast', 'Very Fast', 'Extremely Fast', 'Ultra Fast']
df_clean['tempo_category'] = pd.Categorical(pd.cut(df_clean['tempo'], bins=tempo_bins, labels=tempo_labels))


# Corrected bins for loudness (5 edges, 4 labels)
loudness_bins = [-21, -15, -10, -5, 0]
loudness_labels = ['Very Quiet', 'Quiet', 'Moderate', 'Loud']
df_clean['loudness_category'] = pd.Categorical(pd.cut(df_clean['loudness'], bins=loudness_bins, labels=loudness_labels))
df_clean

,track_name,track_artist,track_popularity,track_album_name,track_album_release_date,playlist_name,playlist_genre,playlist_subgenre,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,popularity_category,tempo_category,loudness_category
0,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66.00000,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,pop,dance pop,0.748,0.916000,...,0.058300,0.102000,0.000000,0.0653,0.509883,122.036,194754.0,Medium Popularity,Fast,Loud
1,Memories - Dillon Francis Remix,Maroon 5,67.00000,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,pop,dance pop,0.726,0.815000,...,0.037300,0.072400,0.004210,0.3570,0.509883,99.972,162600.0,Medium Popularity,Slow,Loud
2,All the Time - Don Diablo Remix,Zara Larsson,42.45689,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,pop,dance pop,0.675,0.931000,...,0.074200,0.079400,0.000023,0.1100,0.613000,124.008,176616.0,Medium-Low Popularity,Fast,Loud
4,Someone You Loved - Future Humans Remix,Lewis Capaldi,69.00000,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,pop,dance pop,0.650,0.833000,...,0.035900,0.080300,0.000000,0.0833,0.509883,123.976,189052.0,Medium Popularity,Fast,Loud
5,Beautiful People (feat. Khalid) - Jack Wins Remix,Ed Sheeran,67.00000,Beautiful People (feat. Khalid) [Jack Wins Remix],2019-07-11,Pop Remix,pop,dance pop,0.675,0.698571,...,0.127000,0.079900,0.000000,0.1430,0.585000,124.982,163049.0,Medium Popularity,Fast,Moderate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32826,Wasted,Tiësto,42.45689,Wasted,2014-04-22,♥ EDM LOVE 2020,edm,progressive electro house,0.645,0.832000,...,0.029400,0.001060,0.002640,0.1990,0.375000,112.028,188371.0,Medium-Low Popularity,Moderate,Moderate
32827,Many Ways - Radio Edit,Ferry Corsten feat. Jenny Wahlstrom,27.00000,Many Ways,2013,♥ EDM LOVE 2020,edm,progressive electro house,0.581,0.640000,...,0.106711,0.026600,0.000000,0.5720,0.288000,128.001,196993.0,Low Popularity,Fast,Moderate
32828,City Of Lights - Official Radio Edit,Lush & Simon,42.00000,City Of Lights (Vocal Mix),2014-04-28,♥ EDM LOVE 2020,edm,progressive electro house,0.428,0.922000,...,0.093600,0.076600,0.000000,0.0668,0.210000,128.170,204375.0,Medium-Low Popularity,Fast,Loud
32830,Sweet Surrender - Radio Edit,Starkillers,14.00000,Sweet Surrender (Radio Edit),2014-04-21,♥ EDM LOVE 2020,edm,progressive electro house,0.529,0.821000,...,0.048100,0.108000,0.000001,0.1500,0.436000,127.989,210112.0,Low Popularity,Fast,Moderate


In [127]:
# release_date had NaT values die niet met dropna() verwijderd konden worden
df_clean['track_album_release_date'] = df_clean['track_album_release_date'].fillna(0)
df_clean = df_clean[df_clean['track_album_release_date'] != '0']
df_clean.describe()

,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
count,20970.000000,20970.000000,20970.000000,20970.000000,20970.000000,20970.000000,20970.000000,20970.000000,20970.000000,20970.000000,20970.000000,20970.000000,20970.000000
mean,42.544820,0.655754,0.700143,5.385418,-6.675162,0.560697,0.106344,0.173166,0.079154,0.189863,0.510701,120.824719,225684.075018
std,24.014587,0.136866,0.170145,3.469171,2.750361,0.476897,0.094576,0.207031,0.206579,0.147298,0.222935,25.635962,55668.825529
min,0.000000,0.168000,0.075000,0.000000,-20.376000,0.000000,0.023900,0.000008,0.000000,0.021900,0.035000,62.520000,73813.000000
25%,27.000000,0.574000,0.597000,2.000000,-7.924000,0.000000,0.042200,0.018100,0.000000,0.095200,0.346000,100.116500,190694.250000
50%,42.456890,0.656000,0.701000,5.386847,-6.425000,1.000000,0.068000,0.099000,0.000031,0.137000,0.509883,120.838811,220940.500000
75%,61.000000,0.752000,0.830000,8.000000,-4.773000,1.000000,0.121000,0.229000,0.019700,0.232000,0.677000,130.104500,248916.750000
max,100.000000,0.964000,0.995000,11.000000,-0.955000,1.000000,0.609000,0.968000,0.955000,0.973000,0.973000,205.958000,504200.000000


In [128]:
df_clean

,track_name,track_artist,track_popularity,track_album_name,track_album_release_date,playlist_name,playlist_genre,playlist_subgenre,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,popularity_category,tempo_category,loudness_category
0,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66.00000,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,pop,dance pop,0.748,0.916000,...,0.058300,0.102000,0.000000,0.0653,0.509883,122.036,194754.0,Medium Popularity,Fast,Loud
1,Memories - Dillon Francis Remix,Maroon 5,67.00000,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,pop,dance pop,0.726,0.815000,...,0.037300,0.072400,0.004210,0.3570,0.509883,99.972,162600.0,Medium Popularity,Slow,Loud
2,All the Time - Don Diablo Remix,Zara Larsson,42.45689,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,pop,dance pop,0.675,0.931000,...,0.074200,0.079400,0.000023,0.1100,0.613000,124.008,176616.0,Medium-Low Popularity,Fast,Loud
4,Someone You Loved - Future Humans Remix,Lewis Capaldi,69.00000,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,pop,dance pop,0.650,0.833000,...,0.035900,0.080300,0.000000,0.0833,0.509883,123.976,189052.0,Medium Popularity,Fast,Loud
5,Beautiful People (feat. Khalid) - Jack Wins Remix,Ed Sheeran,67.00000,Beautiful People (feat. Khalid) [Jack Wins Remix],2019-07-11,Pop Remix,pop,dance pop,0.675,0.698571,...,0.127000,0.079900,0.000000,0.1430,0.585000,124.982,163049.0,Medium Popularity,Fast,Moderate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32826,Wasted,Tiësto,42.45689,Wasted,2014-04-22,♥ EDM LOVE 2020,edm,progressive electro house,0.645,0.832000,...,0.029400,0.001060,0.002640,0.1990,0.375000,112.028,188371.0,Medium-Low Popularity,Moderate,Moderate
32827,Many Ways - Radio Edit,Ferry Corsten feat. Jenny Wahlstrom,27.00000,Many Ways,2013,♥ EDM LOVE 2020,edm,progressive electro house,0.581,0.640000,...,0.106711,0.026600,0.000000,0.5720,0.288000,128.001,196993.0,Low Popularity,Fast,Moderate
32828,City Of Lights - Official Radio Edit,Lush & Simon,42.00000,City Of Lights (Vocal Mix),2014-04-28,♥ EDM LOVE 2020,edm,progressive electro house,0.428,0.922000,...,0.093600,0.076600,0.000000,0.0668,0.210000,128.170,204375.0,Medium-Low Popularity,Fast,Loud
32830,Sweet Surrender - Radio Edit,Starkillers,14.00000,Sweet Surrender (Radio Edit),2014-04-21,♥ EDM LOVE 2020,edm,progressive electro house,0.529,0.821000,...,0.048100,0.108000,0.000001,0.1500,0.436000,127.989,210112.0,Low Popularity,Fast,Moderate


## Feature splitting for release_date (Domain expertise)

In [129]:
# Convert track_album_release_date to datetime, coercing errors
date_month = pd.to_datetime(df_clean['track_album_release_date'], errors='coerce')

# Use .loc to assign the new 'release_month' column safely
df_clean.loc[:, 'release_month'] = date_month.dt.strftime('%B')

df_clean.drop(columns=['instrumentalness', 'acousticness', 'mode', 'playlist_subgenre', 'playlist_name'], inplace=True)
df_clean

,track_name,track_artist,track_popularity,track_album_name,track_album_release_date,playlist_genre,danceability,energy,key,loudness,speechiness,liveness,valence,tempo,duration_ms,popularity_category,tempo_category,loudness_category,release_month
0,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66.00000,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,pop,0.748,0.916000,6.000000,-2.634000,0.058300,0.0653,0.509883,122.036,194754.0,Medium Popularity,Fast,Loud,June
1,Memories - Dillon Francis Remix,Maroon 5,67.00000,Memories (Dillon Francis Remix),2019-12-13,pop,0.726,0.815000,11.000000,-4.969000,0.037300,0.3570,0.509883,99.972,162600.0,Medium Popularity,Slow,Loud,December
2,All the Time - Don Diablo Remix,Zara Larsson,42.45689,All the Time (Don Diablo Remix),2019-07-05,pop,0.675,0.931000,1.000000,-3.432000,0.074200,0.1100,0.613000,124.008,176616.0,Medium-Low Popularity,Fast,Loud,July
4,Someone You Loved - Future Humans Remix,Lewis Capaldi,69.00000,Someone You Loved (Future Humans Remix),2019-03-05,pop,0.650,0.833000,1.000000,-4.672000,0.035900,0.0833,0.509883,123.976,189052.0,Medium Popularity,Fast,Loud,March
5,Beautiful People (feat. Khalid) - Jack Wins Remix,Ed Sheeran,67.00000,Beautiful People (feat. Khalid) [Jack Wins Remix],2019-07-11,pop,0.675,0.698571,8.000000,-5.385000,0.127000,0.1430,0.585000,124.982,163049.0,Medium Popularity,Fast,Moderate,July
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32826,Wasted,Tiësto,42.45689,Wasted,2014-04-22,edm,0.645,0.832000,2.000000,-5.595000,0.029400,0.1990,0.375000,112.028,188371.0,Medium-Low Popularity,Moderate,Moderate,April
32827,Many Ways - Radio Edit,Ferry Corsten feat. Jenny Wahlstrom,27.00000,Many Ways,2013,edm,0.581,0.640000,5.000000,-8.367000,0.106711,0.5720,0.288000,128.001,196993.0,Low Popularity,Fast,Moderate,NaN
32828,City Of Lights - Official Radio Edit,Lush & Simon,42.00000,City Of Lights (Vocal Mix),2014-04-28,edm,0.428,0.922000,5.386847,-1.814000,0.093600,0.0668,0.210000,128.170,204375.0,Medium-Low Popularity,Fast,Loud,April
32830,Sweet Surrender - Radio Edit,Starkillers,14.00000,Sweet Surrender (Radio Edit),2014-04-21,edm,0.529,0.821000,6.000000,-6.722152,0.048100,0.1500,0.436000,127.989,210112.0,Low Popularity,Fast,Moderate,April


In [130]:
df_clean.to_csv("./dataset/spotify_songs_restructured_cleaned.csv", index=False)